# CS5660 Assignment 1: Evolutionary Algorithm for 8-Queens Problem
**Student:** Sidney Thomas

## Problem Statement
Implement an Evolutionary Algorithm to solve the 8-Queens problem using the specified representation, operators, and parameters.


## 1. Imports and Setup


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
from typing import List, Tuple, Dict
import copy
from collections import defaultdict


## 2. Problem Definition and Fitness Function

The 8-Queens problem is formulated as:
- **Objective**: Maximize f(Q) = n(n-1)/2 - A(Q)
- **Constraints**: Each row and column must contain exactly one queen
- **Representation**: Permutation of numbers 1-8 (column positions for rows 1-8)
- **Fitness**: Minimize number of attacking queen pairs (lower is better, 0 = solution)


In [ ]:
def count_attacking_pairs(queen_positions: List[int]) -> int:
    """
    Count the number of attacking pairs of queens.
    
    Args:
        queen_positions: List of column positions for each row (1-8)
        
    Returns:
        Number of attacking pairs
    """
    n = len(queen_positions)
    attacking_pairs = 0
    
    for i in range(n):
        for j in range(i + 1, n):
            # Check if queens attack each other
            # Same row: impossible due to permutation representation
            # Same column: impossible due to permutation representation
            # Same diagonal: |row1 - row2| == |col1 - col2|
            if abs(i - j) == abs(queen_positions[i] - queen_positions[j]):
                attacking_pairs += 1
    
    return attacking_pairs

def fitness_function(queen_positions: List[int]) -> int:
    """
    Calculate fitness of a queen configuration.
    Lower values are better (0 = perfect solution).
    
    Args:
        queen_positions: List of column positions for each row (1-8)
        
    Returns:
        Number of attacking pairs (fitness value)
    """
    return count_attacking_pairs(queen_positions)

def is_solution(queen_positions: List[int]) -> bool:
    """
    Check if the configuration is a valid solution.
    
    Args:
        queen_positions: List of column positions for each row (1-8)
        
    Returns:
        True if no queens attack each other
    """
    return fitness_function(queen_positions) == 0


## 3. EA Components Implementation


### 3.1 Parent Selection (Best 2 out of Random 5)


In [ ]:
def parent_selection(population: List[List[int]], fitness_values: List[int]) -> Tuple[List[int], List[int]]:
    """
    Select two parents using tournament selection.
    Choose 5 individuals randomly and select the best 2.
    
    Args:
        population: List of queen configurations
        fitness_values: Corresponding fitness values
        
    Returns:
        Tuple of two parent configurations
    """
    n = len(population)
    
    # Select 5 random indices
    tournament_indices = random.sample(range(n), 5)
    
    # Get fitness values for tournament participants
    tournament_fitness = [(fitness_values[i], i) for i in tournament_indices]
    
    # Sort by fitness (lower is better)
    tournament_fitness.sort()
    
    # Select best 2
    parent1_idx = tournament_fitness[0][1]
    parent2_idx = tournament_fitness[1][1]
    
    return population[parent1_idx], population[parent2_idx]


### 3.2 Cut-and-Crossfill Crossover


In [ ]:
def cut_and_crossfill_crossover(parent1: List[int], parent2: List[int]) -> Tuple[List[int], List[int]]:
    """
    Implement cut-and-crossfill crossover for permutation representation.
    
    Args:
        parent1: First parent configuration
        parent2: Second parent configuration
        
    Returns:
        Tuple of two offspring configurations
    """
    n = len(parent1)
    
    # Select random crossover point (1 to n-1)
    crossover_point = random.randint(1, n - 1)
    
    # Create children
    child1 = [0] * n
    child2 = [0] * n
    
    # Copy first segment
    child1[:crossover_point] = parent1[:crossover_point]
    child2[:crossover_point] = parent2[:crossover_point]
    
    # Fill second segment of child1 from parent2
    child1_set = set(child1[:crossover_point])
    for value in parent2:
        if value not in child1_set:
            for i in range(crossover_point, n):
                if child1[i] == 0:
                    child1[i] = value
                    child1_set.add(value)
                    break
    
    # Fill second segment of child2 from parent1
    child2_set = set(child2[:crossover_point])
    for value in parent1:
        if value not in child2_set:
            for i in range(crossover_point, n):
                if child2[i] == 0:
                    child2[i] = value
                    child2_set.add(value)
                    break
    
    return child1, child2


### 3.3 Swap Mutation


In [ ]:
def swap_mutation(individual: List[int]) -> List[int]:
    """
    Apply swap mutation by randomly selecting two positions and swapping their values.
    
    Args:
        individual: Queen configuration to mutate
        
    Returns:
        Mutated configuration
    """
    mutated = individual.copy()
    n = len(mutated)
    
    # Select two random positions
    pos1, pos2 = random.sample(range(n), 2)
    
    # Swap values
    mutated[pos1], mutated[pos2] = mutated[pos2], mutated[pos1]
    
    return mutated


### 3.4 Survival Selection (Replace Worst)


In [ ]:
def survival_selection(population: List[List[int]], fitness_values: List[int], 
                      offspring: List[List[int]], offspring_fitness: List[int]) -> Tuple[List[List[int]], List[int]]:
    """
    Select survivors by keeping the best n individuals from (population + offspring).
    
    Args:
        population: Current population
        fitness_values: Current population fitness values
        offspring: New offspring
        offspring_fitness: Offspring fitness values
        
    Returns:
        Tuple of (new_population, new_fitness_values)
    """
    # Combine population and offspring
    combined_population = population + offspring
    combined_fitness = fitness_values + offspring_fitness
    
    # Create list of (fitness, index) pairs
    fitness_index_pairs = [(fitness, i) for i, fitness in enumerate(combined_fitness)]
    
    # Sort by fitness (lower is better)
    fitness_index_pairs.sort()
    
    # Select best n individuals
    n = len(population)
    selected_indices = [pair[1] for pair in fitness_index_pairs[:n]]
    
    new_population = [combined_population[i] for i in selected_indices]
    new_fitness = [combined_fitness[i] for i in selected_indices]
    
    return new_population, new_fitness


## 4. Main Evolutionary Algorithm


In [ ]:
class EightQueensEA:
    """
    Evolutionary Algorithm for solving the 8-Queens problem.
    """
    
    def __init__(self, population_size=100, mutation_prob=0.8, crossover_prob=1.0):
        self.population_size = population_size
        self.mutation_prob = mutation_prob
        self.crossover_prob = crossover_prob
        self.max_evaluations = 10000
        
        # Tracking variables
        self.generation = 0
        self.evaluations = 0
        self.best_fitness_history = []
        self.avg_fitness_history = []
        self.best_individual = None
        self.best_fitness = float('inf')
    
    def initialize_population(self) -> Tuple[List[List[int]], List[int]]:
        """
        Initialize population with random permutations.
        
        Returns:
            Tuple of (population, fitness_values)
        """
        population = []
        fitness_values = []
        
        for _ in range(self.population_size):
            # Create random permutation of 1-8
            individual = list(range(1, 9))
            random.shuffle(individual)
            
            fitness = fitness_function(individual)
            
            population.append(individual)
            fitness_values.append(fitness)
            
            self.evaluations += 1
            
            # Track best individual
            if fitness < self.best_fitness:
                self.best_fitness = fitness
                self.best_individual = individual.copy()
        
        return population, fitness_values
    
    def evolve_generation(self, population: List[List[int]], fitness_values: List[int]) -> Tuple[List[List[int]], List[int]]:
        """
        Evolve one generation of the population.
        
        Args:
            population: Current population
            fitness_values: Current fitness values
            
        Returns:
            Tuple of (new_population, new_fitness_values)
        """
        offspring = []
        offspring_fitness = []
        
        # Generate offspring
        for _ in range(self.population_size // 2):  # 2 offspring per pair
            # Parent selection
            parent1, parent2 = parent_selection(population, fitness_values)
            
            # Crossover
            if random.random() < self.crossover_prob:
                child1, child2 = cut_and_crossfill_crossover(parent1, parent2)
            else:
                child1, child2 = parent1.copy(), parent2.copy()
            
            # Mutation
            if random.random() < self.mutation_prob:
                child1 = swap_mutation(child1)
            if random.random() < self.mutation_prob:
                child2 = swap_mutation(child2)
            
            # Evaluate offspring
            fitness1 = fitness_function(child1)
            fitness2 = fitness_function(child2)
            
            offspring.extend([child1, child2])
            offspring_fitness.extend([fitness1, fitness2])
            
            self.evaluations += 2
            
            # Track best individual
            if fitness1 < self.best_fitness:
                self.best_fitness = fitness1
                self.best_individual = child1.copy()
            if fitness2 < self.best_fitness:
                self.best_fitness = fitness2
                self.best_individual = child2.copy()
        
        # Survival selection
        new_population, new_fitness = survival_selection(population, fitness_values, offspring, offspring_fitness)
        
        return new_population, new_fitness
    
    def run(self) -> Dict:
        """
        Run the evolutionary algorithm.
        
        Returns:
            Dictionary with results and statistics
        """
        # Initialize population
        population, fitness_values = self.initialize_population()
        
        # Record initial statistics
        self.best_fitness_history.append(self.best_fitness)
        self.avg_fitness_history.append(np.mean(fitness_values))
        
        # Main evolution loop
        while self.evaluations < self.max_evaluations and self.best_fitness > 0:
            self.generation += 1
            
            # Evolve one generation
            population, fitness_values = self.evolve_generation(population, fitness_values)
            
            # Record statistics
            self.best_fitness_history.append(self.best_fitness)
            self.avg_fitness_history.append(np.mean(fitness_values))
            
            # Print progress every 100 generations
            if self.generation % 100 == 0:
                print(f"Generation {self.generation}: Best fitness = {self.best_fitness}, Avg fitness = {np.mean(fitness_values):.2f}")
        
        # Final results
        result = {
            'best_individual': self.best_individual,
            'best_fitness': self.best_fitness,
            'generations': self.generation,
            'evaluations': self.evaluations,
            'solved': self.best_fitness == 0,
            'best_fitness_history': self.best_fitness_history,
            'avg_fitness_history': self.avg_fitness_history
        }
        
        return result


## 5. Visualization Functions


In [ ]:
def visualize_chessboard(queen_positions: List[int], title: str = "8-Queens Solution"):
    """
    Visualize the 8-queens configuration on a chessboard.
    
    Args:
        queen_positions: List of column positions for each row (1-8)
        title: Title for the plot
    """
    fig, ax = plt.subplots(figsize=(8, 8))
    
    # Create chessboard pattern
    board = np.zeros((8, 8))
    for i in range(8):
        for j in range(8):
            if (i + j) % 2 == 0:
                board[i][j] = 1
    
    ax.imshow(board, cmap='gray', alpha=0.3)
    
    # Place queens
    for row in range(8):
        col = queen_positions[row] - 1  # Convert to 0-based indexing
        ax.plot(col, row, 'ro', markersize=20, markeredgecolor='black', markeredgewidth=2)
    
    ax.set_xlim(-0.5, 7.5)
    ax.set_ylim(-0.5, 7.5)
    ax.set_xticks(range(8))
    ax.set_yticks(range(8))
    ax.set_xlabel('Column')
    ax.set_ylabel('Row')
    ax.set_title(title)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

def plot_fitness_evolution(results: Dict, title: str = "Fitness Evolution"):
    """
    Plot the evolution of fitness over generations.
    
    Args:
        results: Results dictionary from EA run
        title: Title for the plot
    """
    fig, ax = plt.subplots(figsize=(10, 6))
    
    generations = range(len(results['best_fitness_history']))
    
    ax.plot(generations, results['best_fitness_history'], 'b-', label='Best Fitness', linewidth=2)
    ax.plot(generations, results['avg_fitness_history'], 'r--', label='Average Fitness', linewidth=2)
    
    ax.set_xlabel('Generation')
    ax.set_ylabel('Fitness (Number of Attacking Pairs)')
    ax.set_title(title)
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()


## 6. Experimental Setup and Execution


In [ ]:
# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)

# EA Parameters (as specified in assignment)
POPULATION_SIZE = 100
MUTATION_PROBABILITIES = [0.6, 0.8, 1.0]  # 60%, 80%, 100%
CROSSOVER_PROBABILITY = 1.0  # 100%
MAX_EVALUATIONS = 10000

print("Evolutionary Algorithm Parameters:")
print(f"Population Size: {POPULATION_SIZE}")
print(f"Mutation Probabilities: {MUTATION_PROBABILITIES}")
print(f"Crossover Probability: {CROSSOVER_PROBABILITY}")
print(f"Max Evaluations: {MAX_EVALUATIONS}")
print("\n" + "="*50 + "\n")


### 6.1 Run Experiments with Different Mutation Probabilities


In [ ]:
# Store results for all runs
all_results = {}

for i, mut_prob in enumerate(MUTATION_PROBABILITIES, 1):
    print(f"\n{'='*20} RUN {i} (Mutation Probability = {mut_prob*100}%) {'='*20}")
    
    # Create and run EA
    ea = EightQueensEA(population_size=POPULATION_SIZE, mutation_prob=mut_prob, crossover_prob=CROSSOVER_PROBABILITY)
    results = ea.run()
    
    # Store results
    all_results[f'run_{i}_pm_{int(mut_prob*100)}'] = results
    
    # Print results
    print(f"\nResults for Run {i} (pm = {mut_prob*100}%):")
    print(f"  Best Fitness: {results['best_fitness']}")
    print(f"  Generations: {results['generations']}")
    print(f"  Evaluations: {results['evaluations']}")
    print(f"  Solved: {results['solved']}")
    print(f"  Best Individual: {results['best_individual']}")
    
    # Visualize solution if found
    if results['solved']:
        print("\nSolution found! Visualizing chessboard...")
        visualize_chessboard(results['best_individual'], f"8-Queens Solution (Run {i}, pm={mut_prob*100}%)")
    
    # Plot fitness evolution
    plot_fitness_evolution(results, f"Fitness Evolution - Run {i} (pm={mut_prob*100}%)")


### 6.2 Comparative Analysis


In [ ]:
# Create comparative plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

colors = ['blue', 'red', 'green']
labels = ['60%', '80%', '100%']

for i, (key, results) in enumerate(all_results.items()):
    generations = range(len(results['best_fitness_history']))
    
    # Plot best fitness
    ax1.plot(generations, results['best_fitness_history'], 
             color=colors[i], label=f'pm={labels[i]}', linewidth=2)
    
    # Plot average fitness
    ax2.plot(generations, results['avg_fitness_history'], 
             color=colors[i], label=f'pm={labels[i]}', linewidth=2)

ax1.set_xlabel('Generation')
ax1.set_ylabel('Best Fitness')
ax1.set_title('Best Fitness Evolution Comparison')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.set_xlabel('Generation')
ax2.set_ylabel('Average Fitness')
ax2.set_title('Average Fitness Evolution Comparison')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Summary table
print("\n" + "="*80)
print("SUMMARY OF ALL RUNS")
print("="*80)
print(f"{'Run':<8} {'Mutation%':<10} {'Best Fitness':<12} {'Generations':<12} {'Solved':<8} {'Evaluations':<12}")
print("-"*80)

for i, (key, results) in enumerate(all_results.items()):
    mut_prob = MUTATION_PROBABILITIES[i] * 100
    print(f"{i+1:<8} {mut_prob:<10.0f} {results['best_fitness']:<12} {results['generations']:<12} {results['solved']:<8} {results['evaluations']:<12}")

print("\n" + "="*80)


## 7. Analysis and Discussion


In [ ]:
print("ANALYSIS AND DISCUSSION")
print("="*50)
print("\n1. PROBLEM FORMULATION:")
print("   - The 8-Queens problem is formulated as a constrained optimization problem")
print("   - Objective: Minimize the number of attacking queen pairs")
print("   - Representation: Permutation of column positions (1-8) for each row")
print("   - Constraints: Each row and column contains exactly one queen")

print("\n2. EVOLUTIONARY ALGORITHM COMPONENTS:")
print("   - Parent Selection: Tournament selection (best 2 out of random 5)")
print("   - Crossover: Cut-and-crossfill crossover (100% probability)")
print("   - Mutation: Swap mutation (60%, 80%, 100% probabilities tested)")
print("   - Survival Selection: Replace worst individuals")

print("\n3. EXPERIMENTAL RESULTS:")
print("   - Population size: 100 individuals")
print("   - Maximum evaluations: 10,000")
print("   - Termination: Solution found OR max evaluations reached")

print("\n4. OBSERVATIONS:")
print("   - The algorithm shows typical EA behavior: rapid initial improvement followed by convergence")
print("   - Different mutation probabilities affect exploration vs exploitation balance")
print("   - Higher mutation rates may help escape local optima but can disrupt good solutions")
print("   - The permutation representation naturally satisfies row and column constraints")

print("\n5. CONCLUSION:")
print("   - The Evolutionary Algorithm successfully implements the specified operators")
print("   - The algorithm demonstrates the exploration-exploitation trade-off")
print("   - Results show the effectiveness of the chosen representation and operators")
